# FIT5196 Assessment 1
#### Student Name: DEEPALI VINAY  and MINGLEI WANG
#### Student ID: 30281229 and 26263092

Start Date: 11/08/2019

Finish Date: 25/08/2019

Version: 2.0

Environment: Python 3.7.1 and Jupyter notebook

Libraries used:
* pandas (for dataframe and exporting csv) 
* re (for regular expression)  

## 1. Introduction
The below program extracts the data from an input file in text format, and cleans, transforms and exports the data into CSV and JSON formats.

The input file used here is `group020.txt`, it comprises of 150 patent details. 
The required tasks are the following:

1. Reading `group020.txt` file which contains the details of the patents.
2. Extract the relevent information for each of the patents.
3. Storing/exporting data in a structured csv (`Group020.csv`) and json (`Group020.json`) formats. 

More details for the task will be given in the following sections.

## 2.  Import libraries 

In [1]:
# Code to import libraries 
import re
import pandas as pd

## 3. Parse Group020 File
#### 3.1 Reading the input file

We are using the python file handling feature to open the text file in default read mode and storing the content of the input file in a variable `line` as a string. Also we are opening a `Group020.json` file in write mode to later generate a json output from the given input.


In [2]:
input_file=open('Group020.txt')
line=input_file.read()


In the below cell we are spliting the input text file with `<?xml version="1.0" encoding="UTF-8"?>\n` as a separator and ignoring the first blank element from the resulting list.

In [3]:
data_list=line.split(sep='<?xml version="1.0" encoding="UTF-8"?>\n')[1:]

Here in the next cell we are creating an empty dataframe with the specified column names.

In [4]:
df=pd.DataFrame(columns=['grant_id','patent_title','kind','number_of_claims','inventors','citations_applicant_count',\
                         'citations_examiner_count','claims_text','abstract']) 

Now we are opening a file in write mode which will be used to store output in json format. As observed in the sample output file, we are writing the first character `{` into the json file `Group020.json`.

In [5]:
json_file=open('Group020.json','w')
json_file.write('{');                 

#### 3.2 Extracting data

* In this part of the program we will be extracting all the patents and the relevent data for each of them from the input file. Each of the columns in the dataframe created will be extracted using appropriate regular expressions.

* There are nine columns specified which are grant_id, patent_kind, patent_title, number_of_claims, citations_examiner_count, citations_applicant_count, inventors, claims_text and abstract.

* There is a for loop iterating over each patent and an empty dictionary `dic` is initialised for each iteration, which will hold the column name as the key, and the corresponding value of column as value, before the dictionary is reset in the following iteration.
    
* As soon as a data is fetched and stored into the dictionary, the same data is written into the output json file following the json file format using the python file handling capabilities.

* In the end of the for loop we will append the dictionary to the dataframe, hence after each iteration in the for loop, we will obtain one new row in the dataframe i.e., each iteration will provide details of a new patent id. 

NOTE: Further description of the below code is provided as inline/block comments.
  

In [6]:
# iterating for each elemnent in the data_list
for each in data_list:
    dic={}        # creating an empty dictionary
    
    each=each.replace("&#x2018;","'").replace("&#x2019;","'").replace("&#x2212;","-") # replacing the unicode with a meaningful symbol
    
    
    ''' retrieving grant_id :
        Matching and fetching grant_id only if it consists of zero or more word characters i.e. [a-zA-Z0-9_] 
        between the specified pattern of substrings.
    '''
    
    grant_id=re.search(r'<us-patent-grant.*file="(\w*)-',each)       # searching for the specified pattern
    dic["grant_id"]=grant_id.group(1)                                # updating the dictionary with the value for grant_id
    json_file.write('"'+dic["grant_id"]+'":{')                       # writing the grant_id into json_file 
     
    
    ''' retrieving patent_title :
        Matching and fetching patent_title details consisting of any type of character, 
        appearing zero or more number of times between the specified pattern of substrings.
    '''
    
    patent_title=re.search(r'<invention-title id.*>(.*)</invention-title>',each)   # searching for the specified pattern                                                                              
    dic['patent_title']=patent_title.group(1)                       # updating the dictionary with the value for patent_title
    json_file.write('"patent_title":"'+dic['patent_title']+'",')                   # writing the patent_title into json_file 
    
    
    ''' kind :
        Matching and fetching kind details of patent consisting of any type of character (except \n)
        appearing zero or more number of times between the specified pattern of substrings. 
        Since the metacharacter dot in regex does not include new line character we are explicitly 
        replacing \n with blank.
    '''
    kind=re.search(r'<publication-reference>(.*?)<kind>(.*?)</kind>',each.replace('\n','')) # search for the specified pattern
    
    # creating a lookup dictionary using the observations from the sample input and output provided
    kind_dic={'S1':'Design Patent',\
              'P2':'Plant Patent Grant (no published application) issued on or after January 2, 2001',\
              'P3':'Plant Patent Grant (with a published application) issued on or after January 2, 2001',\
              'E1':'Reissue Patent',\
              'B1':'Utility Patent Grant (no published application) issued on or after January 2, 2001.',\
              'B2':'Utility Patent Grant (with a published application) issued on or after January 2, 2001.'}
    dic['kind']=kind_dic[kind.group(2)]     # using the look up dictionary and updating 'dic' dictionary with value for kind
    json_file.write('"kind":"'+kind_dic[kind.group(2)]+'",')                      # writing the kind into json_file
    
    
    '''number_of_claims :
       Matching and fetching number of claims for each patent, consisting of any type of character, 
       appearing zero or more number of times between the specified pattern of substrings.
    '''
    number_of_claims=re.search(r'<number-of-claims>(.*)</number-of-claims>',each) # searching for the specified pattern
    dic['number_of_claims']=number_of_claims.group(1)                             # update the dictionary with the searched value
    json_file.write('"number_of_claims":'+number_of_claims.group(1)+',')          # writing the number_of_claims into json_file
    
    
     
    ''' inventor :
        Matching pattern and fetching all the inventors of a patent, consisting of any type of character (except \n)
        appearing zero or more number of times between the specified pattern of substrings. 
        Since the metacharacter dot in regex does not include new line character we are explicitly 
        replacing \n with blank.
    '''
    temp_inventor=re.search(r'<inventors>(.*)</inventors>',each.replace('\n','')) # search for the specified pattern
    fname=re.findall(r'<first-name>(.*?)</first-name>',temp_inventor.group(1))    # find all occurences of the pattern 
    lname=re.findall(r'<last-name>(.*?)</last-name>',temp_inventor.group(1))      # find all occurences of the pattern 
   
    if len(fname)==0 & len(lname)==0:          
        dic['inventors']='[NA]'             # if no inventors were found in the search, update the dictionary with [NA]
    else:
        lot=list(zip(fname,lname))          # else create a list of tuples, where each tuple is a (fname,lname) pair
        dic['inventors']='['+','.join([" ".join(list(i)) for i in lot])+']'  # make an appropriate string and update the dictionary
    json_file.write('"inventors":"'+dic['inventors']+'",')                   # writing the inventors into json_file
    
    
    '''citations_applicant_count & citations_examiner_count :
       Finding the count of citations by applicant and examiner of a patent. First getting both the citations 
       by matching the pattern consisting of any type of character (except \n) appearing zero or more number 
       of times between the specified pattern of substrings and then counting the occurances of each citation type.
       Since the metacharacter dot in regex does not include new line character we are explicitly 
       replacing \n with blank.
    '''
    temp_citation=re.search(r'<us-references-cited>(.*)</us-references-cited>', each.replace('\n',''))
    #count citations_applicant
    if temp_citation :
        applicant_count=len(re.findall(r'cited by applicant',temp_citation.group(1)))
    else:
        applicant_count=0
    #count citations_examiner    
    if temp_citation:
        examiner_count=len(re.findall(r'cited by examiner',temp_citation.group(1)))
    else:
        examiner_count=0
    dic['citations_applicant_count']=applicant_count
    json_file.write('"citations_applicant_count":'+str(applicant_count)+',')
    #store two counts to dic
    dic['citations_examiner_count']=examiner_count
    json_file.write('"citations_examiner_count":'+str(examiner_count)+',')
    
    
    '''claim_text '''
    
    raw_text=re.search(r'<claims id="claims">(.*)</claims>', each.replace('\n','')) #search for the specified pattern
    
    #if a pattern was found in the search
    if raw_text:
        claim_list=raw_text.group(1).strip().split('</claim>')    # split at the occurences of '</claim>' into a list
        claims=[re.sub(r'<.*?>','',j) for j in claim_list][:-1]   # remove tags for each element of the list
        claims_text="["+','.join(claims)+"]"                      # change list to a comma separated string
        dic['claims_text']=claims_text                            # updating the dictionary for claims_text
    
    #if no patterns found in the search
    else:
        dic['claims_text']='[NA]'                                 # updating the dictionary with '[NA]' for claims_text
        
    json_file.write('"claims_text":"'+dic['claims_text']+'",')    # writing the value to json_file
   
    
    '''abstract '''
    
    temp_abstract=re.search(r'<abstract id="abstract">(.*)</abstract>',each.replace('\n','')) # search for the specified pattern
    
    #if a pattern was found in the search
    if temp_abstract:
        abstract_data=temp_abstract.group(1)
        abstract=re.sub(r'<.*?>','',abstract_data)  # removing the tags, by substituting the pattern with nothing
        dic['abstract']=abstract                    # updating the dictionary with the value for abstract
    
    #if no patterns found in the search
    else:
        dic['abstract']='NA'                        # updating the dictionary with 'NA' for abstract
    json_file.write('"abstract":"'+dic['abstract']+'"},') 
        
    df=df.append(dic, ignore_index=True)            # append dictionary to dateframe and ignore index

#### 3.3 Creating csv and json output files
We convert the dataframe to csv format and create a file named `Group020.csv`.

In [7]:
df.to_csv('Group020.csv', index=False)        # creating csv file and ignoring dataframe index

Finally, we complete the json file with `}` and close the earlier opened files.

In [8]:
# finding the current cursor position and replacing the last character (comma) with '}'
json_file.seek(json_file.tell()-1)   
json_file.write('}')

# closing the files
json_file.close() 
input_file.close()

# 3. Summary

This assessment tested the overall python coding knowledge, working with different data types(integers, dictionaries, lists, strings etc), pandas, regex and 
finally documenting skills. We understood, how meaningful data can be fetched from a file with raw data by studying/analysing the patterns and extracting the right information.

**regex** : regular expression was extensively used to search for a particular pattern/substring

**pandas** : pandas was used to organise data in a dataframe and then export it to a csv file succesfully

**file operations**: Python file operations were used to write the data to a json file as observed in the sample output